In [53]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window 

In [63]:
spark = SparkSession.builder.master("local[*]").appName("ETL").getOrCreate()



24/06/07 02:11:33 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 1045956 ms exceeds timeout 120000 ms
24/06/07 02:11:34 WARN SparkContext: Killing executors is not supported by current scheduler.
24/06/07 02:28:41 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:101)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:85)
	at org.apache.spark.storage.BlockManagerMaster.registerBlockManager(BlockManagerMaster.scala:80)
	at org.apache.spark.storage.BlockManager.reregister(BlockManager.scala:642)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1223)
	at 

In [55]:
df = spark.read \
    .format("parquet") \
    .load("hdfs://localhost:9000/raw_zone/fact/activity/vtrhigobertighwo58w9g.parquet")

student_schema =StructType([
    StructField("student_code",LongType()),
    StructField("name",StringType()),
    ])

df1= spark.read \
    .format("csv") \
    .option("header","false") \
    .schema(student_schema) \
    .load("hdfs://localhost:9000/raw_zone/fact/activity/danh_sach_sv_de.csv")
df.printSchema()
df1.printSchema()

root
 |-- student_code: long (nullable = true)
 |-- activity: string (nullable = true)
 |-- numberofFile: long (nullable = true)
 |-- timestamps: string (nullable = true)

root
 |-- student_code: long (nullable = true)
 |-- name: string (nullable = true)



In [58]:

windowspec = Window.partitionBy("student_name","timestamps").orderBy("totalFile")
df2 = df.join(broadcast(df1),df.student_code == df1.student_code,"inner") \
    .select(df.timestamps, \
            df.student_code, \
            df1.name.alias("student_name"), \
            df.activity, \
            df.numberofFile.alias("totalFile")) \
    .withColumn("rank", rank().over(windowspec)) \
    .drop("rank")
df2.show(30)

+----------+------------+--------------------+--------+---------+
|timestamps|student_code|        student_name|activity|totalFile|
+----------+------------+--------------------+--------+---------+
| 6/10/2024|           4|      Trần Trung Anh|   write|        7|
| 6/12/2024|          33|     Nguyễn Bá Thiêm|    read|        5|
| 6/13/2024|          33|     Nguyễn Bá Thiêm| execute|        1|
| 6/15/2024|           6|  Nguyễn Vũ Hòa Bình|   write|        6|
| 6/12/2024|          24|     Nguyễn Hoài Nam| execute|        8|
| 6/12/2024|          22|       Huỳnh Tấn Lộc|   write|        2|
| 6/13/2024|          31|         Tạ Đức Tiến|   write|        9|
| 6/13/2024|           8|        Đỗ Thành Đạt|   write|        4|
| 6/12/2024|          21| Trần Phúc Mạnh Linh|    read|        5|
| 6/10/2024|          26|     Nguyễn Duy Hưng| execute|        2|
| 6/12/2024|          24|     Nguyễn Hoài Nam|    read|       10|
| 6/15/2024|          10|    Nguyễn Quốc Dũng|    read|        6|
| 6/14/202

In [52]:
df2.write.mode("overwrite").csv("data/output.csv")